In [79]:
import requests
import json
from time import sleep

end = 1689983940 #end of training period
span = 8640000 #100 days in seconds (span of each request)

def read_api_key():
    '''Reads in API key from project folder'''
    with open("Stock Price Miner/key.txt") as f:
        key = f.read()
    return key

def read_companies():
    '''Reads list of companies from Data folder.'''
    with open("Stock Price Miner/Data/companies.txt") as f:
        companies = f.read()
    companies = companies.split('\n')
    return companies

def get_data_from_url(url):
    '''Return json data from the passed in URL'''
    for i in range(0,3):
        try:
            return requests.get(url).json()
        except Exception:
            print('Failed to get data. Retrying...')
            sleep(1)
    print('All attempts failed for URL.')
    return []
    
def get_percent_change(start, stop):
    '''Converts the start and stop price into a percent change in price'''
    return round((stop-start)/start, 4)

def generate_url(ticker, key, start, stop):
    '''Generate the URL for the passed in ticker data for the given time range'''
    return f'https://eodhistoricaldata.com/api/intraday/{ticker}.US?api_token={key}&interval=1m&fmt=json&from={start}&to={stop}'

def get_last_2000_days_of_stock_info(ticker, key):
    '''gets the last 2000 days of stock information for the given ticker'''
    data = []
    print(f'Downloading data for {ticker}')
    for i in range(20,0,-1):
        #print(f'{ticker} time slices remaining: {i}')
        start = end - (i * span)
        stop = end - ((i-1) * span) - 60
        data = data + get_data_from_url(generate_url(ticker, key, start, stop))
        sleep(.5)
    json_data = json.dumps({ticker: data}, indent=4)
    with open(f'Stock Price Miner/Data/{ticker}.json', 'w') as out:
        out.write(json_data)
    print(f'{ticker} completed.')
    return data

def test_company_data():
    '''Call the API for each company to make sure it gets a 200 response.'''
    companies = read_companies()
    key = read_api_key()
    data_returned = {'success':[], 'failed':[]}
    ctr = 0
    for c in companies:
        try: 
            data = requests.get(generate_url(c, key, end-60, end))
            data_returned['success'].append({c: str(data)})
            print(c, ':', str(data))
        except Exception as e:
            data_returned['failed'].append({c: 'Exception'})
            print(c, e)
        sleep(.5)
    return data_returned